# Neeps - Resit

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app15-4")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@f8ba6d2

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@602aa07d

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val ut_staff = hiveCxt.table("sqlzoo.ut_staff")
val ut_student = hiveCxt.table("sqlzoo.ut_student")
val ut_event = hiveCxt.table("sqlzoo.ut_event")
val ut_room = hiveCxt.table("sqlzoo.ut_room")
val ut_attends = hiveCxt.table("sqlzoo.ut_attends")
val ut_teaches = hiveCxt.table("sqlzoo.ut_teaches")
val ut_occurs = hiveCxt.table("sqlzoo.ut_occurs")
val ut_modle = hiveCxt.table("sqlzoo.ut_modle")
val ut_week = hiveCxt.table("sqlzoo.ut_week")

ut_staff: DataFrame = [id: string, name: string]
ut_student: DataFrame = [id: string, name: string ... 2 more fields]
ut_event: DataFrame = [id: string, modle: string ... 5 more fields]
ut_room: DataFrame = [id: string, name: string ... 2 more fields]
ut_attends: DataFrame = [student: string, event: string]
ut_teaches: DataFrame = [staff: string, event: string]
ut_occurs: DataFrame = [event: string, week: string]
ut_modle: DataFrame = [id: string, name: string]
ut_week: DataFrame = [id: string, wkstart: string]

## 1.
**Give the day and the time of the event co72002.L01.**

In [5]:
(ut_event.filter(ut_event("id") === "co72002.L01")
 .select("dow", "tod")
 .showHTML())

dow,tod
Monday,10:00


## 2.
**For each event in module co72003 show the day, the time and the place.**

In [6]:
(ut_event.filter(ut_event("modle") === "co72003")
 .select("id", "dow", "tod", "room")
 .showHTML())

id,dow,tod,room
co72003.L01,Monday,14:00,cr.B12


## 3.
**List the id of the events taught by "Chisholm, Ken".**

In [7]:
(ut_event.withColumnRenamed("id", "event")
 .join(ut_teaches, "event")
 .join(ut_staff.filter(ut_staff("name") === "Chisholm, Ken")
       .withColumnRenamed("id", "staff"), "staff")
 .select("event")
 .showHTML())

event
co22006.L01
co22006.L02
co22006.L03
co22006.T01
co22006.T02
co72013.L01
co72013.L02
co72013.T01
co72013.T02
co72013.T06


## 4.
**List the staff who teach in cr.SMH.**

In [8]:
(ut_staff.withColumnRenamed("id", "staff")
 .join(ut_teaches, "staff")
 .join(ut_event.filter(ut_event("room") === "cr.SMH")
        .withColumnRenamed("id", "event"), "event")
 .select("name")
 .distinct()
 .orderBy("name")
 .showHTML())

name
"Barclay, Ken"
"Chisholm, Ken"
"Cumming, Andrew"
"Greig, Frank"
"Hastie, Colin"
"Jackson, Jim"
"Kemmer, Rob"
"Lawson, Alistair"
"Macaulay, Catriona"
"Maclean, Aileen"


## 5.
**Show the total number of hours (over the whole semester) of classes for com.IS.a**

In [9]:
(ut_event.withColumnRenamed("id", "event")
 .join(ut_occurs, "event")
 .join(ut_attends.filter(col("student") === "com.IS.a"),
       "event")
 .groupBy()
 .agg(sum("duration").alias("hrs"))
 .showHTML())

hrs
60


In [10]:
spark.stop()